<a href="https://colab.research.google.com/github/JisungJang-AI-BIO/LAIDD_Developability/blob/main/Boltz-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 환경 설정: Boltz 및 의존성 패키지 설치

`boltz==2.1.1`의 공식 요구사항(`pytorch-lightning==2.5.0`)과 호환되는 PyTorch 버전(`2.3.0`)을 명시하여 모든 패키지를 한 번에 설치합니다.

In [8]:
# 1단계: 기존 패키지를 모두 제거하여 깨끗한 상태를 보장합니다.
print("에러 발생 시 기존 PyTorch 및 Boltz 관련 패키지를 제거합니다...")
!pip uninstall torch torchvision torchaudio boltz -y -q

# 2단계: 모든 제약조건(boltz, torch==2.2.0 등)을 단일 명령어로 전달하여 한 번에 설치합니다.
# --extra-index-url을 사용하여 PyPI와 PyTorch 서버를 모두 검색하도록 합니다.
print("\nBoltz와 호환되는 모든 패키지를 동시에 설치합니다...")
!pip install boltz torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --extra-index-url https://download.pytorch.org/whl/cu121 -q

# 3단계: 설치된 버전을 최종 확인하여, torch가 의도한 대로 2.2.0으로 설치되었는지 검증합니다.
print("\n설치된 패키지 버전을 확인합니다.")
!pip show torch torchvision

기존 PyTorch 및 Boltz 관련 패키지를 제거합니다...

Boltz와 호환되는 모든 패키지를 동시에 설치합니다...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

---

## 2. Trastuzumab Fab 단편 구조 예측 (실행 예제)

항원 없이 Trastuzumab 항체의 가변 영역(VH, VL)만으로 Fab 구조를 예측합니다. Colab 환경에서 정상적으로 실행됩니다.

In [3]:
import yaml
import datetime
import os
import glob
import py3Dmol

# 입력 데이터: Trastuzumab Fab (VH, VL)
heavy_chain_sequence = "EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRWGGDGFYAMDYWGQGTLVTVSS"
light_chain_sequence = "DIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSRSGTDFTLTISSLQPEDFATYYCQQHYTTPPTFGQGTKVEIK"
output_name = "Trastuzumab_Fab_prediction"

# YAML 파일 생성
input_data = {
    'sequences': [
        {'protein': {'id': 'H', 'sequence': heavy_chain_sequence}},
        {'protein': {'id': 'L', 'sequence': light_chain_sequence}}
    ],
    'name': output_name
}
yaml_path = f"{output_name}.yaml"
with open(yaml_path, 'w') as f:
    yaml.dump(input_data, f, indent=2, sort_keys=False)
print(f"✅ YAML 파일 생성 완료: {yaml_path}")

# Boltz 예측 실행
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = f"boltz_run_{timestamp}"
print(f"✅ 결과 저장 폴더: {output_dir}")
print("\nBoltz-2 예측을 시작합니다...")
!boltz predict {yaml_path} --use_msa_server --output_format pdb --out_dir {output_dir}
print("\n--- 예측 프로세스 완료 ---")

# 결과 시각화
print("\n결과 시각화를 시작합니다...")
pdb_files = glob.glob(os.path.join(output_dir, '**', '*.pdb'), recursive=True)
try:
    if not pdb_files:
        raise FileNotFoundError(f"'{output_dir}' 폴더에서 .pdb 파일을 찾을 수 없습니다.")
    pdb_files.sort()
    pdb_file_path = pdb_files[0]
    print(f"✅ 발견된 PDB 파일: {pdb_file_path}")
    with open(pdb_file_path, 'r') as f:
        pdb_data = f.read()
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, "pdb")
    view.setStyle({'chain':'H'}, {'cartoon': {'color':'blue'}})
    view.setStyle({'chain':'L'}, {'cartoon': {'color':'green'}})
    view.zoomTo()
    view.show()
except FileNotFoundError as e:
    print(f"오류: {e}")
except Exception as e:
    print(f"시각화 중 오류가 발생했습니다: {e}")

ModuleNotFoundError: No module named 'py3Dmol'

---

## 3. (참고) Trastuzumab - HER2 항원 결합 구조 예측

⚠️ **경고**: 이 예제는 항원의 크기가 매우 커서, 일반 Colab GPU 환경에서는 **메모리 부족(Out of Memory)으로 실패할 가능성이 매우 높습니다.** 더 높은 사양의 GPU 환경에서 테스트하거나 참고용으로만 활용하는 것을 권장합니다.

In [ ]:
import yaml
import datetime
import os
import glob
import py3Dmol

# 입력 데이터: Trastuzumab(VH, VL)과 HER2 항원
heavy_chain_sequence_orig = "EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRWGGDGFYAMDYWGQGTLVTVSS"
light_chain_sequence_orig = "DIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSRSGTDFTLTISSLQPEDFATYYCQQHYTTPPTFGQGTKVEIK"
antigen_sequence_orig = "TQVCTGTDMKLRLPASPETHLDMLRHLYQGCQVVQGNLELTYLPTNASLSFLQDIQEVQGYVLIAHNQVRQVPLQRLRIVRGTQLFEDNYALAVLDNGDPLNNTTPVTGASPGGLRELQLRSLTEILKGGVLIQRNPQLCYQDTILWKDIFHKNNQLALTLIDTNRSRACHPCSPMCKGSRCWGESSEDCQSLTRTVCAGGCARCKGPLPTDCCHEQCAAGCTGPKHSDCLACLHFNHSGICELHCPALVTYNTDTFESMPNPEGRYTFGASCVTACPYNYLSTDVGSCTLVCPLHNQEVTAEDGTQRCEKCSKPCARVCYGLGMEHLREVRAVTSANIQEFAGCKKIFGSLAFLPESFDGDPASNTAPLQPEQLQVFETLEEITGYLYISAWPDSLPDLSVFQNLQVIRGRILHNGAYSLTLQGLGISWLGLRSLRELGSGLALIHHNTHLCFVHTVPWDQLFRNPHQALLHTANRPEDECVGEGLACHQLCARGHCWGPGPTQCVNCSQFLRGQECVEECRVLQGLPREYVNARHCLPCHPECQPQNGSVTCFGPEADQCVACAHYKDPPFCVARCPSGVKPDLSYMPIWKFPDEEGACQPCPINCTHSCVDLDDKGCPAEQRASPLT"
output_name_orig = "Trastuzumab_HER2_binding_prediction"

# YAML 파일 생성
input_data_orig = {
    'sequences': [
        {'protein': {'id': 'H', 'sequence': heavy_chain_sequence_orig}},
        {'protein': {'id': 'L', 'sequence': light_chain_sequence_orig}},
        {'protein': {'id': 'A', 'sequence': antigen_sequence_orig}}
    ],
    'name': output_name_orig
}
yaml_path_orig = f"{output_name_orig}.yaml"
with open(yaml_path_orig, 'w') as f:
    yaml.dump(input_data_orig, f, indent=2, sort_keys=False)
print(f"✅ YAML 파일 생성 완료: {yaml_path_orig}")

# Boltz 예측 실행
timestamp_orig = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir_orig = f"boltz_run_{timestamp_orig}"
print(f"✅ 결과 저장 폴더: {output_dir_orig}")
print("\nBoltz-2 예측을 시작합니다...")
!boltz predict {yaml_path_orig} --use_msa_server --output_format pdb --out_dir {output_dir_orig}
print("\n--- 예측 프로세스 완료 ---")

# 결과 시각화
print("\n결과 시각화를 시작합니다...")
pdb_files_orig = glob.glob(os.path.join(output_dir_orig, '**', '*.pdb'), recursive=True)
try:
    if not pdb_files_orig:
        raise FileNotFoundError(f"'{output_dir_orig}' 폴더에서 .pdb 파일을 찾을 수 없습니다.")
    pdb_files_orig.sort()
    pdb_file_path_orig = pdb_files_orig[0]
    print(f"✅ 발견된 PDB 파일: {pdb_file_path_orig}")
    with open(pdb_file_path_orig, 'r') as f:
        pdb_data_orig = f.read()
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data_orig, "pdb")
    view.setStyle({'chain':'H'}, {'cartoon': {'color':'blue'}})
    view.setStyle({'chain':'L'}, {'cartoon': {'color':'green'}})
    view.setStyle({'chain':'A'}, {'cartoon': {'color':'red'}})
    view.zoomTo()
    view.show()
except FileNotFoundError as e:
    print(f"오류: {e}")
except Exception as e:
    print(f"시각화 중 오류가 발생했습니다: {e}")